In [1]:
from src.harness import architecture as arch
from src.harness import utils

import copy
import functools
import numpy as np
import tensorflow as tf
from tensorflow import keras
from typing import Any, Callable, Dict, Iterable, List, Literal, Tuple

2024-11-01 13:06:11.481232: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-01 13:06:11.515619: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-01 13:06:11.524997: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-01 13:06:11.544532: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-01 13:06:14.542410: W tensorflow/compiler/tf2

In [2]:
# Typedefs
Genome = List[tf.Tensor]
GenomeInit = Callable[[keras.Model], Genome]
GenomeMetricCallback = Callable[[Dict, Genome], Any]

class Individual:
    # One shared copy throughout the class (all individuals in population share same original weights)
    ARCHITECTURE = None
    MODEL = None
    DATA = None
    
    def __init__(
        self, 
        architecture: arch.Architecture, 
        fitness_function: Callable[[Literal['Individual']], float],
        genome_init: GenomeInit,
    ):
        # If this is the first instance of the class, initialize it with read only copies of data
        if self.ARCHITECTURE is None:
            self.ARCHITECTURE = architecture
            self.MODEL = self.ARCHITECTURE.get_model_constructor()()
            self.DATA = self.ARCHITECTURE.load_data()
            
        self.genome = genome_init(self.model)
        self.rng = np.random.default_rng()
        self.fitness_function = fitness_function
        self.metrics = {}
        
    @staticmethod
    def copy_from(individual: Literal['Individual']) -> Literal['Individual']:
        copied = copy.deepcopy(individual)
        copied.metrics.clear()
        copied.rng = np.random.default_rng()
        return copied
        
    @property
    def fitness(self) -> float | None:
        return self.fitness_function(self)
    
    @property
    def architecture(self) -> arch.Architecture | None:
        return self.ARCHITECTURE
    
    @property
    def model(self) -> keras.Model | None:
        return self.MODEL
    
    @property
    def data(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray] | None:
        return self.DATA
    
    @property
    def training_data(self) -> Tuple[np.ndarray, np.ndarray] | None:
        if self.data is not None:
            X_train, _, Y_train, _ = self.data
            return X_train, Y_train
        
    @property
    def test_data(self) -> Tuple[np.ndarray, np.ndarray] | None:
        if self.data is not None:
            _, X_test, _, Y_test = self.data
            # For now, use a smaller portion for proof of concept
            return X_test[:100], Y_test[:100]
        
    def copy_model(self) -> keras.Model | None:
        if self.model is not None:
            return copy.deepcopy(self.model)
        
    def sample_mask(self) -> Genome:
        return [
            tf.cast(
                np.random.uniform(low=0, high=1, size=probabilities.shape)
                > probabilities,
                tf.float32,
            )
            for probabilities in self.genome
        ]
    
    @staticmethod
    def eval_sample_masks(individual: Literal['Individual'], num_evaluations: int = 5):
        model = individual.copy_model()
        model.compile(
            loss=keras.losses.CategoricalCrossentropy(),
            metrics=[keras.metrics.CategoricalAccuracy()]
        )
        X_test, Y_test = individual.test_data
        
        losses = np.zeros(num_evaluations)
        accuracies = np.zeros(num_evaluations)
        for i in range(num_evaluations):
            mask = individual.sample_mask()
            model.set_weights([w * m for w, m in zip(individual.model.get_weights(), mask)])
            losses[i], accuracies[i] = model.evaluate(X_test, Y_test)
        individual.metrics["mean_loss"] = np.mean(losses)
        individual.metrics["mean_accuracy"] = np.mean(accuracies)
        
    @staticmethod
    def eval_mask(individual: Literal['Individual']):
        model = individual.copy_model()
        model.compile(
            loss=keras.losses.CategoricalCrossentropy(),
            metrics=[keras.metrics.CategoricalAccuracy()]
        )
        X_test, Y_test = individual.test_data
        loss, accuracy = model.evaluate(X_test, Y_test)
        individual.metrics["loss"] = loss
        individual.metrics["accuracy"] = accuracy
    
    # Mutation Methods
    
    # Indirect Encoding Mutations
    
    @staticmethod
    def decrease_prob(individual: Literal['Individual'], rate: float, scale: float):
        for layer_index, layer in enumerate(individual.genome):
            perturb_mask = (np.random.uniform(
                low=0, 
                high=1, 
                size=layer.shape,
            ) > rate).astype(np.int8)
            perturbations = -np.abs(individual.rng.normal(
                loc=0,
                scale=scale,
                size=layer.shape,
            )) * perturb_mask
            individual.genome[layer_index] = np.clip(
                layer + perturbations, 
                a_min=0,
                a_max=1,
            )
            
    @staticmethod
    def resample_prob(individual: Literal['Individual'], rate: 0.05):
        for layer_index, layer in enumerate(individual.genome):
            perturb_mask = np.random.uniform(
                low=0, 
                high=1, 
                size=layer.shape,
            ) > rate
            resampled = np.random.uniform(
                low=0, 
                high=1, 
                size=layer.shape,
            )
            np.place(layer, perturb_mask, resampled)
            
    # Direct Encodings Mutations
            
    @staticmethod
    def prune(individual: Literal['Individual'], rate: float):
        for layer_index, layer in enumerate(individual.genome):
            perturb_mask = np.random.uniform(
                low=0, 
                high=1, 
                size=layer.shape,
            ) > rate
            individual.genome[layer_index][perturb_mask] = 0
            
    @staticmethod
    def unprune(individual: Literal['Individual'], rate: float):
        for layer_index, layer in enumerate(individual.genome):
            perturb_mask = np.random.uniform(
                low=0, 
                high=1, 
                size=layer.shape,
            ) > rate
            individual.genome[layer_index][perturb_mask] = 1
    
    # Initialization methods
    
    @staticmethod
    def init_ones(model: keras.Model) -> Genome:
        return [
            np.ones_like(
                weights,
                dtype=np.float32,
            )
            for weights in model.get_weights()
        ]
    
    @staticmethod
    def random_prob(model: keras.Model) -> Genome:
        return [
            np.random.uniform(low=0, high=1, size=weights.shape)
            for weights in model.get_weights()
        ]
    
    # Crossover methods
    
    @staticmethod
    def layer_crossover(p1: Literal['Individual'], p2: Literal['Individual']) -> Iterable[Literal['Individual']]:
        child1, child2 = list(map(Individual.copy_from, (p1, p2)))
        p1_weights = p1.genome
        p2_weights = p2.genome
        parents = np.random.randint(low=0, high=2, size=len(p1_weights))
        child1.genome = copy.deepcopy([
            p1_weights[layer_index] if parent == 0 
            else p2_weights[layer_index] 
            for layer_index, parent in enumerate(parents)
        ])
        child2.genome = copy.deepcopy([
            p2_weights[layer_index] if parent == 0 
            else p1_weights[layer_index] 
            for layer_index, parent in enumerate(parents)
        ])
        return child1, child2
    
    @staticmethod
    def neuron_crossover(p1: Literal['Individual'], p2: Literal['Individual']) -> Iterable[Literal['Individual']]:
        child1, child2 = list(map(Individual.copy_from, (p1, p2)))
        p1_weights = p1.genome
        p2_weights = p2.genome
        for layer_index, weights in enumerate(p1_weights):
            # Generate a 0/1 for each row, then extend it across all outgoing synapses
            parents = np.repeat(
                np.random.randint(low=0, high=2, size=weights.shape[0]),
                1 if weights.ndim == 1 else weights.shape[1],
                axis=0,
            ).reshape((weights.shape))
            inverse_parents = np.logical_not(parents).astype(np.int8)
            
            # This multiplication uses masks to perform selection
            child1.genome[layer_index] = p1_weights[layer_index] * parents \
                + p2_weights[layer_index] * inverse_parents
            child2.genome[layer_index] = p2_weights[layer_index] * parents \
                + p1_weights[layer_index] * inverse_parents
        return child1, child2
    
    @staticmethod
    def synapse_crossover(p1: Literal['Individual'], p2: Literal['Individual']) -> Iterable[Literal['Individual']]:
        child1, child2 = list(map(Individual.copy_from, (p1, p2)))
        p1_weights = p1.genome
        p2_weights = p2.genome
        for layer_index, weights in enumerate(p1_weights):
            # Generate a 0/1 for each row, then extend it across all outgoing synapses
            parents = np.random.randint(low=0, high=2, size=weights.shape)
            inverse_parents = np.logical_not(parents).astype(np.int8)
            
            # This multiplication uses masks to perform selection
            child1.genome[layer_index] = p1_weights[layer_index] * parents \
                + p2_weights[layer_index] * inverse_parents
            child2.genome[layer_index] = p2_weights[layer_index] * parents \
                + p1_weights[layer_index] * inverse_parents
        return child1, child2

# Typedefs
Mutation = Callable[[Individual], None]
Crossover = Callable[[Individual, Individual], Individual]
FitnessFunction = Callable[[Individual], float]


In [3]:
# (µ + λ) strategy with tournament selection
def evolutionary_algorithm(
    num_generations: int,
    num_parents: int,
    num_children: int,
    tournament_size: int,
    num_tournament_winners: int,
    individual_constructor: Callable[[], Individual],
    fitness_eval: FitnessFunction,
    crossover: Crossover | None = None,
    mutations: List[Mutation] = [],
    genome_metric_callbacks: List[GenomeMetricCallback] = [],
):
    if num_tournament_winners > tournament_size:
        raise ValueError("Cannot have more tournament winners than participants")
        
    best_solution = None
    best_fitness = -np.inf
    genome_metrics = {"best_solution_fitness": np.zeros(num_generations)}
    
    # Create and evaluate the initial population
    population = []
    for _ in range(num_parents):
        individual = individual_constructor()
        fitness_eval(individual)
        if individual.fitness > best_fitness:
            best_solution = copy.deepcopy(individual)
            best_fitness = best_solution.fitness
        population.append(individual)
    
    for generation_index in range(num_generations):
        children = []
        while len(children) < num_children:
            parents = np.random.choice(population, 2)
            new_children = crossover(*parents) if crossover else list(map(Individual.copy_from, parents))
            
            for child in new_children:
                for mutation in mutations:
                    mutation(child)
                fitness_eval(child)
                if child.fitness > best_fitness:
                    best_solution = child
                    best_fitness = best_solution.fitness
            children.extend(new_children)
        population.extend(children)
        
        # Seed next generation with best solution found thus far
        next_generation = [best_solution]
        while len(next_generation) < num_parents:
            tournament = sorted(
                np.random.choice(population, size=tournament_size),
                key = lambda x: x.fitness,
                reverse=True,
            )
            next_generation.extend(tournament[:num_tournament_winners])
        population = next_generation
        
        for callback in genome_metric_callbacks:
            callback(genome_metrics, population)
        genome_metrics["best_solution_fitness"][generation_index] = best_fitness
        
    return genome_metrics
    

In [4]:
# Callbacks for genome metrics

def average_sparsity_p(data: Dict, population: List[Individual]):
    def sum_genome_params(genome: Genome) -> int:
        return sum(map(lambda layer: layer.size, genome))
    def sum_genome_probabilities(genome: Genome) -> float:
        return sum(map(lambda layer: np.sum(layer), genome))
    def sum_over_genomes(f: Callable[[Genome], float]) -> float:
        return sum(map(lambda x: f(x.genome), population))
    
    total_sum_params = sum_over_genomes(sum_genome_params)
    total_sum_probabilities = sum_over_genomes(sum_genome_probabilities)
    
    key = "average_sparsity_p"
    if data.get(key) is None:
        data[key] = []
    data[key].append(total_sum_probabilities / total_sum_params)

def dummy_callback(data: Dict, *args):
    if data.get("dummy") is None:
        data["dummy"] = []
    data["dummy"].append(len(data["dummy"]))

In [5]:
# Sampling Mask (Indirect Encoding)

num_runs = 1

individual_constructor = functools.partial(
    Individual, 
    architecture=arch.Architecture('lenet', 'mnist'), 
    fitness_function=lambda x: x.metrics.get('mean_accuracy'),
    genome_init=Individual.random_prob,
)
fitness_eval = functools.partial(Individual.eval_sample_masks, num_evaluations=1)

mutations = [
    functools.partial(Individual.resample_prob, rate=1e-5),
    functools.partial(Individual.decrease_prob, rate=1e-4, scale=2.5e-2),
]
    
genome_metric_callbacks = [
    dummy_callback,
    average_sparsity_p,
]
kwargs = {
    "num_generations": 10,
    "num_parents": 10,
    "num_children": 10,
    "tournament_size": 4,
    "num_tournament_winners": 2,
    "individual_constructor": individual_constructor,
    # Construction kwargs
    "fitness_eval": fitness_eval,
    "mutations": mutations,
    "crossover": Individual.layer_crossover,
    "genome_metric_callbacks": genome_metric_callbacks,
}

indirect_encoding_metrics = []
for _ in range(num_runs):
    indirect_encoding_metrics.append(evolutionary_algorithm(**kwargs))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - categorical_accuracy: 0.0155 - loss: 2.3353  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.1209 - loss: 2.2906  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - categorical_accuracy: 0.1603 - loss: 2.2866  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.0890 - loss: 2.3135  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.1055 - loss: 2.3197  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.0367 - loss: 2.3329      
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.1421 - loss: 2.2967  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.0799 - loss: 2.2856  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.0705 - loss: 2.3067  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.2137 - loss: 2.3032  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.0677 - loss: 2.3171  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 

In [ ]:
# Flipping Mask (Direct Encoding)

num_runs = 5

individual_constructor = functools.partial(
    Individual, 
    architecture=arch.Architecture('lenet', 'mnist'), 
    fitness_function=lambda x: x.metrics.get('accuracy'),
    genome_init=Individual.init_ones,
)
fitness_eval = functools.partial(Individual.eval_mask)

mutations = [
    functools.partial(Individual.unprune, rate=1e-5),
    functools.partial(Individual.prune, rate=1e-4),
]
    
genome_metric_callbacks = [
    # dummy_callback,
    average_sparsity_p,
]
kwargs = {
    "num_generations": 50,
    "num_parents": 50,
    "num_children": 50,
    "tournament_size": 4,
    "num_tournament_winners": 2,
    "individual_constructor": individual_constructor,
    # Construction kwargs
    "fitness_eval": fitness_eval,
    "mutations": mutations,
    "crossover": Individual.synapse_crossover,
    "genome_metric_callbacks": genome_metric_callbacks,
}

direct_encoding_metrics = []
for _ in range(num_runs):
    direct_encoding_metrics.append(evolutionary_algorithm(**kwargs))
    

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.0617 - loss: 2.3673  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.0987 - loss: 2.3218  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.0716 - loss: 2.4044  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.1169 - loss: 2.3275  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.0738 - loss: 2.3499  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.0771 - loss: 2.3605  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.0798 - loss: 2.4032  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.1091 - loss: 2.3295  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.0307 - loss: 2.3370  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.1242 - loss: 2.2978  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.1383 - loss: 2.3141  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.07

In [ ]:
print(
    list(
        map(lambda x: x.get('average_sparsity_p'), direct_encoding_metrics)
    )
)

In [13]:
best_solution_fitnesses = np.array(
    [d.get('best_solution_fitness') for d in direct_encoding_metrics]
)
max_fitness = np.max(best_solution_fitnesses)
averaged_accuracies = np.mean(best_solution_fitnesses, axis=0)

[[0.18000001 0.18000001 0.18000001 0.18000001 0.18000001 0.18000001
  0.18000001 0.18000001 0.18000001 0.18000001]]


TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [13]:
best_solution_fitnesses = np.array(
    [d.get('best_solution_fitness') for d in direct_encoding_metrics]
print(np.array(best_solution_fitnesses))
averaged_accuracies = np.mean(best_solution_fitnesses, axis=0)

[[0.18000001 0.18000001 0.18000001 0.18000001 0.18000001 0.18000001
  0.18000001 0.18000001 0.18000001 0.18000001]]


TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [7]:
# Directly masking off everything but the top 20% magnitude
# Doesn't perform any better than random chance.
# Not surprising.

a = arch.Architecture('lenet', 'mnist')
X_train, X_test, Y_train, Y_test = a.load_data()
constructor = a.get_model_constructor()

# Just create masks off initial weight magnitude
cutoff_prop = 0.2
fitness_values = []
for i in range(10):
    utils.set_seed(i)
    model = constructor()
    weights = model.get_weights()
    sorted_weights = [np.sort(w, axis=None) for w in weights]
    cutoff_indices = [int((cutoff_prop / 2) * w.size) for w in sorted_weights]
    cutoff_values = [
        (w[cutoff], w[-cutoff]) 
        for w, cutoff in zip(sorted_weights, cutoff_indices)
    ]
    masked_weights = [
        w * ((w <= low_cutoff) | (w >= high_cutoff)).astype(np.int8)
        for w, (low_cutoff, high_cutoff)
        in zip(weights, cutoff_values)
    ]

    model.set_weights(masked_weights)
    model.compile(
            loss=keras.losses.CategoricalCrossentropy(),
            metrics=[keras.metrics.CategoricalAccuracy()]
        )
    fitness_values.append(model.evaluate(X_test, Y_test))
    

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - categorical_accuracy: 0.1328 - loss: 2.2989
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - categorical_accuracy: 0.0755 - loss: 2.3219
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - categorical_accuracy: 0.1024 - loss: 2.3063
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - categorical_accuracy: 0.1286 - loss: 2.2852
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - categorical_accuracy: 0.1090 - loss: 2.3109
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - categorical_accuracy: 0.1069 - loss: 2.3195
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - categorical_accuracy: 0.1044 - loss: 2.3169
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - categorical_accuracy: 0.0889 - loss: 2.3193
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - categorical_accuracy: 0.1160 - loss: 2.3085
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - categorical_accuracy: 0.1136 - loss: 2.3058


In [8]:
print(f"Mean accuracy: {np.mean(list(map(lambda tup: tup[1], fitness_values))):.2%}")

Mean accuracy: 10.79%
